I'm gonna overwrite a lot of this notebook's old content. I changed the way I'm calculating wt, and wanna test that my training worked. 

In [35]:
from pearce.emulator import *
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [36]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [37]:
from GPy.models import GPKroneckerGaussianRegression
from GPy.kern import *
import h5py

In [38]:
from pearce.emulator import GPKroneckerGaussianRegressionVar

In [39]:
training_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_v3/PearceXiggCosmo.hdf5'
test_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_test_v3/PearceXiggCosmoTest.hdf5'

In [ ]:
accs = []
log_accs = []
noise = []
for rbin in xrange(18):

    f = h5py.File(training_file, 'r')

    Ys = []
    Yerrs = []
    for i in xrange(40):
        Ys.append(f['cosmo_no_%02d'%i]['a_1.000']['obs'].value[:, rbin])
        Yerrs.append(np.sqrt(f['cosmo_no_%02d'%i]['a_1.000']['cov'].value[:, rbin, rbin]) )

    n_hods = 500
    start_idx = 0
    X1 = f.attrs['cosmo_param_vals']
    X2 = f.attrs['hod_param_vals'][start_idx:start_idx+n_hods]
    Y = np.vstack(Ys)[:, start_idx:start_idx+n_hods]
    Yerr = np.vstack(Yerrs)[:, start_idx:start_idx+n_hods]
    
    noise.append([Y, Yerr])

    f.close()
    # how to add training errors?

    K1 =RBF(input_dim=7, ARD = True)#+ Linear(input_dim = 7, ARD = False) + Bias(input_dim=7)# + White(input_dim=7)
    K2 =  RBF(input_dim=4, ARD = True)#+ Linear(input_dim = 4, ARD = False) + Bias(input_dim=4)# + White(input_dim=4)

    model = GPKroneckerGaussianRegressionVar(X1, X2, Y, Yerr, K1, K2)#, noise_var = 0.01 )

    model.optimize_restarts(num_restarts=10, verbose = False)

    print K1.param_array
    print K2.param_array
    print 
    
    f2 = h5py.File(test_file, 'r')

    Y2s = []
    for i in xrange(35):
        Y2s.append(f2['cosmo_no_%02d'%i]['a_1.000']['obs'].value[:, rbin])

    testX1 = f2.attrs['cosmo_param_vals']
    testX2 = f2.attrs['hod_param_vals'][:100]
    testY = np.vstack(Y2s)[:, :100]
    f2.close()

    predY, _ = model.predict(testX1, testX2)
    med_acc, mean_acc = np.median( np.abs( (10**predY[:,0] - 10**testY.flatten(order='F'))/10**testY.flatten(order='F') )  ), \
np.mean( np.abs( (10**predY[:,0] - 10**testY.flatten(order='F'))/10**testY.flatten(order='F') )  ) 
    print rbin, med_acc, mean_acc
    accs.append((med_acc, mean_acc))
    log_accs.append((np.median( np.abs( (predY[:,0] - testY.flatten(order='F'))/testY.flatten(order='F') )  ), \
np.mean( np.abs( (predY[:,0] - testY.flatten(order='F'))/testY.flatten(order='F') )  ) ))
    print

[  4.1875946    0.7861257    0.09096075   1.25945749   0.3858755
   2.19498549  13.11296506   5.2140262 ]
[ 4.31553397  2.58307519  3.32970487  2.94324899  3.12898886]

0 0.0360296277174 0.0828087102657

[  5.96397402   0.79367287   0.06758834   1.38283777   0.2952396
   3.57659657  19.71057111   7.51808948]
[ 5.99029061  3.01728234  3.7187364   3.52569533  3.43875991]

1 0.033327659787 0.0832584060462

[  2.93755297   1.7306073    0.09089783   1.13741317   0.62346443
   1.70514628  13.62309509   6.21232485]
[ 4.23478411  2.84403919  2.83779638  2.72970834  2.56060293]

2 0.0218037845372 0.0443178156073

[  2.81088367   2.68872541   0.03200131   2.10183679   1.40490504
   1.16863481  15.30200698   8.24276067]
[ 3.05829425  2.83377835  2.68704266  2.69179941  2.1229928 ]

3 0.0244803820073 0.0318192305362

[  5.65883996   0.25878396   0.03462683   3.33572204   2.97390277
   2.54353229  28.94779894  17.56524212]
[ 4.88418074  3.06297647  1.25050904  3.3961178   2.30028909]

4 0.015645770

In [ ]:
accs = np.array(accs)

In [ ]:
f = h5py.File(training_file, 'r')
#print f.attrs.keys()
r_bins = f.attrs['scale_bins']
f.close()
rpoints = (r_bins[1:]+r_bins[:-1])/2.0

In [ ]:
plt.plot(rpoints, accs[:,0], label = 'Median Error')
plt.plot(rpoints, accs[:,1], label = 'Mean Error')

plt.ylim([0.001, 0.5])
plt.ylabel('Perc. Accuracy')
plt.xlabel(r'$r$ [Mpc]')
plt.legend(loc = 'best')
plt.loglog();